In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import LinearSVC, SVC
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from sklearn.metrics import classification_report, f1_score
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier, KernelDensity
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/weather predicting/train_data.csv')
test = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/weather predicting/test_data.csv')

y, train = train['RainTomorrow'], train.drop(['RainTomorrow'], axis=1)

In [ ]:
print(train.shape)
train.head()

(135414, 21)


,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,Brisbane,13.7,23.6,0.0,5.0,9.6,SSE,24.0,SSW,ESE,7.0,9.0,65.0,53.0,1028.4,1026.7,1.0,1.0,18.3,22.3,0
1,Sydney,15.4,21.2,61.2,5.8,2.7,S,43.0,WSW,SSW,19.0,24.0,91.0,65.0,1018.8,1016.5,8.0,7.0,15.6,20.7,1
2,Richmond,16.8,30.9,1.0,NaN,NaN,NE,35.0,NNW,NE,7.0,26.0,80.0,38.0,1020.4,1013.2,1.0,NaN,19.6,30.0,0
3,NorfolkIsland,13.9,17.1,0.2,3.8,8.9,SE,37.0,SE,SSE,17.0,20.0,56.0,63.0,1027.7,1026.1,3.0,1.0,15.3,15.8,0
4,Dartmoor,7.4,14.4,1.8,2.4,8.3,SSW,37.0,NaN,SW,0.0,17.0,100.0,58.0,1026.0,1026.1,NaN,NaN,9.1,13.5,1


In [ ]:
print(test.shape)
test.head()

(10046, 21)


,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,CoffsHarbour,15.6,26.1,0.0,NaN,NaN,E,22.0,SSW,E,9.0,11.0,77.0,68.0,1018.7,1018.2,NaN,5.0,21.9,25.5,0
1,SalmonGums,9.2,26.5,0.0,NaN,NaN,S,33.0,SW,SW,13.0,9.0,65.0,29.0,NaN,NaN,NaN,NaN,18.5,25.9,0
2,Ballarat,5.8,15.3,3.4,NaN,NaN,SSW,59.0,WSW,SSW,17.0,26.0,100.0,51.0,1016.7,1017.3,8.0,6.0,7.2,14.1,1
3,PerthAirport,7.7,20.8,0.0,3.8,1.9,NNE,33.0,ENE,ENE,17.0,15.0,33.0,25.0,1026.9,1023.8,7.0,7.0,14.7,20.4,0
4,Bendigo,4.8,20.0,0.2,NaN,NaN,WNW,48.0,NaN,W,0.0,28.0,98.0,64.0,1021.9,1019.4,2.0,8.0,8.9,16.4,0


#Preprocessing

In [ ]:
features = train.columns[:-1]

In [ ]:
num_features = []
for feature in features:
  if feature not in ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']:
    num_features.append(feature)

In [ ]:
for feature in ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']:
  train[feature].fillna('NO', inplace=True)
  test[feature].fillna('NO', inplace=True)


for feature in num_features:
  train[feature].fillna(train[feature].mean(), inplace=True)
  test[feature].fillna(test[feature].mean(), inplace=True)

In [ ]:
one_hot_train = pd.get_dummies(train, columns = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])
one_hot_test = pd.get_dummies(test, columns = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])

In [ ]:
one_hot_train.columns

Index(['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm',
       ...
       'WindDir3pm_NO', 'WindDir3pm_NW', 'WindDir3pm_S', 'WindDir3pm_SE',
       'WindDir3pm_SSE', 'WindDir3pm_SSW', 'WindDir3pm_SW', 'WindDir3pm_W',
       'WindDir3pm_WNW', 'WindDir3pm_WSW'],
      dtype='object', length=117)

In [ ]:
one_hot_test.columns

Index(['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm',
       ...
       'WindDir3pm_NO', 'WindDir3pm_NW', 'WindDir3pm_S', 'WindDir3pm_SE',
       'WindDir3pm_SSE', 'WindDir3pm_SSW', 'WindDir3pm_SW', 'WindDir3pm_W',
       'WindDir3pm_WNW', 'WindDir3pm_WSW'],
      dtype='object', length=117)

# Визуализация

In [ ]:
!pip install umap-learn

In [ ]:
import umap

umap = umap.UMAP(n_neighbors=5, min_dist=0.7)
transformed_features = umap.fit_transform(one_hot_train[:10000])

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
draw_vectors(
    transformed_features[:, 0], 
    transformed_features[:, 1], 
    color=[["red", "blue"][t] for t in y]
    )

Figure(id='1003', ...)

In [ ]:
scaler = StandardScaler()
scaler.fit(one_hot_train)

StandardScaler()

In [ ]:
parameters = {'C':[0.01, 0.1, 1., 10., 100.]}
log = LogisticRegression(random_state=123, class_weight='balanced', solver='saga', C=0.1)
clf = GridSearchCV(log, parameters, cv=5, n_jobs=4)
clf.fit(one_hot_train, y)

best_params = clf.best_params_
pprint(best_params)

In [ ]:
#.set_params(**best_params)
model = LogisticRegression(random_state=123, class_weight='balanced', solver='saga', C=0.1)
model.fit(pd.DataFrame(scaler.transform(one_hot_train), columns=scaler.feature_names_in_), y)
print("Train f1-score - ", f1_score(model.predict(pd.DataFrame(scaler.transform(one_hot_train), columns=scaler.feature_names_in_)), y, average='macro'))

Train f1-score -  0.7412030910154438


In [ ]:
pd.DataFrame(model.predict(pd.DataFrame(scaler.transform(one_hot_test), columns=scaler.feature_names_in_)), columns=['y']).to_csv('weather_predicting.csv', index=False, header=None)